In [57]:
import pandas as pd
from scipy.stats import t
import numpy as np
from scipy.stats import chi2
from scipy.stats import ttest_ind
from scipy.stats import f

P1. Tesla Batteries: This problem uses the file teslaData.csv that contains the survey data for the
battery depreciation in various Tesla vehicles.Use the data in the column “Remaining battery range” for the following:Calculate the sample mean and sample standard deviation for the remaining
range on the Tesla vehicles from the Tesla users who provided the data.

In [58]:
tesla_data = pd.read_csv('teslaData-sys501.csv')
tesla_data.head()

,Region,Manufacture date,Vehicle model,Daily Usage,Lifetime average,Date you charged to 100%,Your range at 100% [mi],100% range for this trim level [mi],Remaining battery capacity,Mileage [mi],Mileage [km]
0,Europe (excluding UK) & Asia Pacific,14-Apr-15,Model S 85,Medium,225.0,10-Jun-15,249,245,101.52%,2113.0,3400.0
1,UK,20-Jun-16,Model S 85,Medium,318.0,24-Dec-16,242,245,98.78%,16800.0,27037.0
2,Europe (excluding UK) & Asia Pacific,22-Aug-16,Model S 90D,Medium,210.0,18-Jan-18,268,278,96.42%,15845.0,25500.0
3,Canada,20-Jul-13,Model S P85,Mild,176.0,23-Aug-17,260,266,97.66%,37352.0,60113.0
4,Europe (excluding UK) & Asia Pacific,24-Jun-16,Model S 75,Mild,188.0,8-Oct-16,237,233,101.60%,821.0,1322.0


In [59]:
if tesla_data['Remaining battery capacity'].dtype == 'O': 
    tesla_data['Remaining battery capacity'] = tesla_data['Remaining battery capacity'].str.rstrip('%').astype('float') / 100

range_mean = tesla_data['Remaining battery capacity'].mean()
range_std = tesla_data['Remaining battery capacity'].std()
range_mean, range_std


(0.9646714890613973, 0.02928036991328033)

Both normal and t distribution would give very similar results since we have a large
number of data points. However, t-distribution is more appropriate since the
population standard deviation is unknown.

Calculate 95% confidence interval and 99% confidence interval for the
population mean for the remaining battery capacity.

In [60]:
alpha_95 = 1 - 0.95
alpha_99 = 1 - 0.99

t_95 = t.ppf(1 - alpha_95 / 2, df_range)
t_99 = t.ppf(1 - alpha_99 / 2, df_range)

margin_95 = t_95 * (range_std / np.sqrt(df_range + 1))
margin_99 = t_99 * (range_std / np.sqrt(df_range + 1))

confidence_95 = (range_mean - margin_95, range_mean + margin_95)
confidence_99 = (range_mean - margin_99, range_mean + margin_99)

confidence_95, confidence_99

((0.9631456416566924, 0.9661973364661023),
 (0.9626651955090034, 0.9666777826137912))

 Estimate the population variance with 95% confidence for the remaining
range.

In [61]:
alpha_95 = 0.05

chi2_lower_95 = chi2.ppf(alpha_95 / 2, df_range)
chi2_upper_95 = chi2.ppf(1 - (alpha_95 / 2), df_range)
civ95_lower = ((df_range + 1) * sample_variance_percentage) / chi2_upper_95
civ95_upper = ((df_range + 1) * sample_variance_percentage) / chi2_lower_95
civ95 = (civ95_lower, civ95_upper)

civ95

(7.991678909553455, 9.260628270116289)

P2. An astronomer is interested in measuring the distance, in light-years, from his observatory
to a distant star. Each measurement does not yield the exact distance due to atmospheric
conditions and measurement errors, but merely an estimate. As a result, the astronomer
plans to make a series of measurements and then use the average value of these
measurements as his estimated value of the actual distance. If the astronomer believes
that the values of the measurements are independent and identically distributed random
variables having a common mean d (the actual distance) and a common variance of 4
(light-years2), how many measurements need he make to be reasonably sure that his
estimated distance is accurate to within ±0.5 light-year?

In [62]:
sigma = 2  
ME = 0.5  

z_score_95 = 1.96  
n_required = (z_score_95 * sigma / ME) ** 2
n_required


61.465599999999995

In [63]:
z_score_99 = 2.576  
n_required_99 = (z_score_99 * sigma / ME) ** 2
n_required_99


106.17241600000001

P3. Gasoline Prices: Use this data to determine whether there is a statistically significant difference in gas
prices in these two cities.

In [64]:
gasoline_data = pd.read_csv('gasoline_data.csv')
gasoline_data

,City 1,City 2
0,2.42,2.32
1,2.53,2.06
2,2.48,1.68
3,2.26,2.84
4,2.35,2.85
5,2.63,2.88
6,2.58,2.53
7,2.48,2.14
8,2.43,2.46
9,2.22,2.58


H0: 𝜇1 = 𝜇2 

Ha: 𝜇1 ≠ 𝜇2

Null hypothesis states that the average gas prices in both cities are the same and
the alternative hypothesis states that there is a statistically significant difference
between them.

In [65]:
var_city1 = gasoline_data['City 1'].var()
var_city2 = gasoline_data['City 2'].var()

var_ratio = var_city2 / var_city1
var_city1, var_city2, var_ratio


(0.013799583333333322, 0.2311818713450292, 16.752815339474942)

Since the ratio of the larger one to the smaller one is a large number (16.75), the
test with unequal variances is more appropriate.

In [66]:
# Welch's t-test
t_stat_welch, p_value_welch = ttest_ind(
    gasoline_data['City 1'].dropna(),
    gasoline_data['City 2'].dropna(),
    equal_var=False 
)

t_stat_welch, p_value_welch

(-1.2232997543071713, 0.2350803363631529)

This p-value indicates that there is not a statistically significant difference in the average gas prices between the two cities

In [70]:
F_statistic = var_city2 / var_city1

dfn = gasoline_data['City 2'].dropna().count() - 1 
dfd = gasoline_data['City 1'].dropna().count() - 1  
p_value_f_test = f.cdf(F_statistic, dfn, dfd)
F_statistic, p_value_f_test


(16.752815339474942, 0.9999992383562499)

the results suggest that there is a statistically
significant difference in the variances for gas prices in both cities.